In [1]:
import numpy as np
import pandas as pd


C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\XA155XB\Anaconda3\envs\walmart2\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
train = pd.read_csv("../walmart_sales/train_payload2_azureml v3.4.csv")
test = pd.read_csv("../walmart_sales/test_payload2_azureml v3.4.csv")

In [3]:
train["Date"] = pd.to_datetime(train["Date"])
test["Date"] = pd.to_datetime(test["Date"])

In [16]:
df = pd.concat([train, test]).set_index(["Region", "Date"]).sort_index(ascending=True).reset_index()

In [17]:
print(df[df["Region"]==1].to_markdown())

|     |   Region | Date                |   Weekly_Sales |   Temperature |   Fuel_Price |     CPI |   Unemployment |   Size |
|----:|---------:|:--------------------|---------------:|--------------:|-------------:|--------:|---------------:|-------:|
|   0 |        1 | 2010-02-05 00:00:00 |    1.64369e+06 |         42.31 |        2.572 | 211.096 |          8.106 | 151315 |
|   1 |        1 | 2010-02-12 00:00:00 |    1.64196e+06 |         38.51 |        2.548 | 211.242 |          8.106 | 151315 |
|   2 |        1 | 2010-02-19 00:00:00 |    1.61197e+06 |         39.93 |        2.514 | 211.289 |          8.106 | 151315 |
|   3 |        1 | 2010-02-26 00:00:00 |    1.40973e+06 |         46.63 |        2.561 | 211.32  |          8.106 | 151315 |
|   4 |        1 | 2010-03-05 00:00:00 |    1.55481e+06 |         46.5  |        2.625 | 211.35  |          8.106 | 151315 |
|   5 |        1 | 2010-03-12 00:00:00 |    1.43954e+06 |         57.79 |        2.667 | 211.381 |          8.106 | 151315 |


In [15]:
df.to_csv("../walmart_sales/sample_demand.csv", index=False)

## Getting Unique Region & DMZ

In [4]:
df = pd.read_csv("~/Documents/EY Internship/walmart_sales/demand_sql.csv")


In [17]:
df1 = df.groupby(["Region", "DMZ"]).size().reset_index().rename(columns={0: "count"})

In [20]:
df1 = df1.drop(["count"], axis=1)

In [22]:
df1.to_excel("~/Documents/EY Internship/walmart_sales/dmz.xlsx", index=False)

## Sum by Region

In [63]:
train_df = pd.read_csv("~/Documents/EY Internship/walmart_sales/demand_sql.csv")

In [64]:
train_df.head()

,Region,DMZ,Date,Weekly_Sales
0,1,1,5/2/10,24924.50
1,1,1,12/2/10,46039.49
2,1,1,19/2/10,41595.55
3,1,1,26/2/10,19403.54
4,1,1,5/3/10,21827.90


In [67]:
train_df["Date"]= pd.to_datetime(train_df["Date"], format="%d/%m/%y")

In [68]:
train_df

,Region,DMZ,Date,Weekly_Sales
0,1,1,2010-02-05,24924.50
1,1,1,2010-02-12,46039.49
2,1,1,2010-02-19,41595.55
3,1,1,2010-02-26,19403.54
4,1,1,2010-03-05,21827.90
...,...,...,...,...
117599,12,1182,2012-09-28,593.29
117600,12,1182,2012-10-05,491.87
117601,12,1182,2012-10-12,570.87
117602,12,1182,2012-10-19,384.34


In [69]:
train_region = train_df.groupby(["Region", "Date"]).sum().drop(["DMZ"], axis=1).reset_index()
train_region.sort_values(by=["Region", "Date"])

,Region,Date,Weekly_Sales
0,1,2010-02-05,1643690.90
1,1,2010-02-12,1641957.44
2,1,2010-02-19,1611968.17
3,1,2010-02-26,1409727.59
4,1,2010-03-05,1554806.68
...,...,...,...
1711,12,2012-09-28,880415.67
1712,12,2012-10-05,979825.92
1713,12,2012-10-12,934917.47
1714,12,2012-10-19,960945.43


In [72]:
# train_region = train_df.groupby(["Region", "Date"]).sum().drop(["DMZ"], axis=1)
train_region["Date"]

0      2010-02-05
1      2010-02-12
2      2010-02-19
3      2010-02-26
4      2010-03-05
          ...    
1711   2012-09-28
1712   2012-10-05
1713   2012-10-12
1714   2012-10-19
1715   2012-10-26
Name: Date, Length: 1716, dtype: datetime64[ns]

In [71]:
train_region.to_csv("~/Documents/EY Internship/walmart_sales/train_region_v2.csv", index=False)

## Train Test Split

In [5]:
df = pd.read_csv("~/Documents/EY Internship/walmart_sales/train_region_v2.csv")

""


In [28]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()


for region in pd.unique(df["Region"]):
    temp = df.loc[df["Region"] == region, ]
    temp_train = temp.iloc[:-(int(0.2*len(temp))), :]
    temp_test = temp.iloc[-(int(0.2*len(temp))):, :]
    train_df = pd.concat([train_df, temp_train], axis = 0)
    test_df = pd.concat([test_df, temp_test], axis = 0)

In [32]:
# train_df[train_df["Region"] ==8]

In [35]:
train_df.to_csv("~/Documents/EY Internship/walmart_sales/train_payload1.csv", index=False)
test_df.to_csv("~/Documents/EY Internship/walmart_sales/test_payload1.csv", index=False)

## Test Dataset

In [8]:
test_df = pd.read_csv("~/Documents/EY Internship/walmart_sales/test.csv")
test_df.head()
test_df["Date"]= pd.to_datetime(test_df["Date"], format="%d/%m/%y")
test_df
test_region = test_df.groupby(["Region", "Date"]).sum().drop(["DMZ"], axis=1).reset_index()
test_region
# train_region.sort_values(by=["Region", "Date"])
# # train_region = train_df.groupby(["Region", "Date"]).sum().drop(["DMZ"], axis=1)
# train_region["Date"]
# train_region.to_csv("~/Documents/EY Internship/walmart_sales/train_region_v2.csv", index=False)

,Region,Date,IsHoliday,DMZ.1
0,1,2012-11-02,0,3182
1,1,2012-11-09,0,3182
2,1,2012-11-16,0,3137
3,1,2012-11-23,72,3236
4,1,2012-11-30,0,3281
...,...,...,...,...
463,12,2013-06-28,0,2938
464,12,2013-07-05,0,2938
465,12,2013-07-12,0,2862
466,12,2013-07-19,0,2862


In [10]:
test_region.to_csv("../walmart_sales/test_region1.csv", index=False)